In [ ]:
import numpy as np
import pandas as pd

df_orders = pd.read_csv('olist_orders_dataset.csv')
df_reviews = pd.read_csv('olist_order_reviews_dataset.csv')
df_items = pd.read_csv('olist_order_items_dataset.csv')
df_products = pd.read_csv('olist_products_dataset.csv')
df_sellers = pd.read_csv('olist_sellers_dataset.csv')
df_customers = pd.read_csv('olist_customers_dataset.csv')
df_translation = pd.read_csv('product_category_name_translation.csv')

df_main = pd.merge(df_orders, df_customers, on='customer_id')

df_main = pd.merge(df_main, df_reviews, on='order_id')

df_main = pd.merge(df_main, df_items, on='order_id')

df_main = pd.merge(df_main, df_products, on='product_id')

df_main = pd.merge(df_main, df_sellers, on='seller_id')

df_main = pd.merge(df_main, df_translation, on='product_category_name')

print(f"--- Birleştirme Tamamlandı! Ana Tablo Boyutu: {df_main.shape} ---")

tarih_sutunlari = [
    'order_purchase_timestamp',
    'order_delivered_customer_date',
    'order_estimated_delivery_date'
]
for col in tarih_sutunlari:
    df_main[col] = pd.to_datetime(df_main[col], errors='coerce')

df_main['delivery_time_days'] = (df_main['order_delivered_customer_date'] - df_main['order_purchase_timestamp']).dt.days
df_main['estimated_time_days'] = (df_main['order_estimated_delivery_date'] - df_main['order_purchase_timestamp']).dt.days
df_main['delivery_delta_days'] = df_main['estimated_time_days'] - df_main['delivery_time_days']

columns_to_keep = [
    'order_id',
    'review_score',
    'delivery_delta_days',
    'product_category_name_english',
    'seller_state',
    'customer_state'
]

df_bi_data = df_main[columns_to_keep]

df_bi_data = df_bi_data.dropna()

print(f"Temizlik sonrası Tableau için hazır satır sayısı: {len(df_bi_data)}")

df_bi_data = df_bi_data.rename(columns={
    'order_id': 'OrderID',
    'review_score': 'Puan',
    'delivery_delta_days': 'GecikmeGunu',
    'product_category_name_english': 'UrunKategorisi',
    'seller_state': 'SaticiEyaleti',
    'customer_state': 'MusteriEyaleti'
})

yeni_dosya_adi = 'olist_tableau_hazir_veri.csv'
df_bi_data.to_csv(yeni_dosya_adi, index=False)

print("\nYeni Sütun Adları:")
print(df_bi_data.head())

--- Birleştirme Tamamlandı! Ana Tablo Boyutu: (110750, 36) ---
Temizlik sonrası Tableau için hazır satır sayısı: 108457

Yeni Sütun Adları:
                            OrderID  Puan  GecikmeGunu UrunKategorisi  \
0  e481f51cbdc54678b7cc49136f2d6af7     4          7.0     housewares   
1  53cdb2fc8bc7dce0b6741e2150273451     4          6.0      perfumery   
2  47770eb9100c2d0c44946d9cf07ec65d     5         17.0           auto   
3  949d5b44dbf5de918fe9c16f97b45f8a     5         13.0       pet_shop   
4  ad21c59c0840e6cb83a9ceb5573f8159     5         10.0     stationery   

  SaticiEyaleti MusteriEyaleti  
0            SP             SP  
1            SP             BA  
2            SP             GO  
3            MG             RN  
4            SP             SP  
